In [1]:
!pip install deepctr_torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 73.6 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.1.1
    Uninstalling typing_extensions-4.1.1:
      Successfully uninstalled typing_extensions-4.1.1
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: h5py
    Found existing installation: h5py 3.7.0
    Uninstalling h5py-3.7.0:
      Successfully uninstalled h5py-3.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.10.1
    Uninstalling tensorboard-2.10.1:
      Successfully uninstalled tensorboard-2.10.1
ERROR: pip's de

In [2]:
import pandas as pd  
from sklearn.preprocessing import LabelEncoder  
import gc,os
import torch  
from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names  
from deepctr_torch.models import *

In [3]:
train_data = pd.read_csv("../input/byyali/training_dataset.csv")  
train_data["buffer_rate"] = train_data["buffer_rate"]*100  
test_data = pd.read_csv("../input/byyali/test_dataset.csv")  
feature_embedding = pd.concat([train_data,test_data]).nunique()  
gc.collect()

35

In [4]:
labs = {}  
colss = ["inner_network_rtt",'domain_name','node_name',"avg_fbt_time","tcp_conntime","icmp_rtt","io_await_avg","io_await_max"]  
for col in colss:  
    lab = LabelEncoder()  
    lab.fit(train_data[col].values.tolist()+test_data[col].values.tolist())  
    labs[col] = lab  
for col in colss:  
    train_data[col] = labs[col].transform(train_data[col])  
    test_data[col] = labs[col].transform(test_data[col])

In [5]:
categorical_feature = colss  
numeric_feature = [cols for cols in train_data.columns if cols not in categorical_feature+["buffer_rate"]]  
  
dense_feature = [cols for cols in train_data.columns if cols not in categorical_feature+["buffer_rate"]]  
sparse_feature = categorical_feature  
test_dense_feature = [cols for cols in test_data.columns if cols not in categorical_feature]  
test_sparse_feature = categorical_feature

In [6]:
fixlen_feature_columns = [SparseFeat(feat,feature_embedding[feat])  
                          for feat in sparse_feature] + [DenseFeat(feat,1) for feat in dense_feature]  
  
dnn_feature_columns = fixlen_feature_columns  
linear_feature_columns = fixlen_feature_columns  

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [7]:
train_model_input = {name:train_data[name] for name in feature_names}  
test_model_input = {name:test_data[name] for name in feature_names}

In [8]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')  
print('device',device)

device cuda


In [9]:
model = DeepFM(linear_feature_columns,dnn_feature_columns,task='regression',  
  dnn_dropout=0.4,seed=42,l2_reg_dnn=0.2,device=device,  
  dnn_activation="relu",dnn_hidden_units=(128,128))  
  
model.compile(  
    optimizer=torch.optim.AdamW(model.parameters(),lr=0.001, weight_decay=1e-5), # 3e-4  
  loss='mae',  
  metrics=['mse','mae']  
)  
  
history = model.fit(train_model_input, train_data["buffer_rate"].values, 
                    batch_size=2048*2,  
                    epochs=100,
                    verbose=1,
                    validation_split=0.2,
                    shuffle=True)  

cuda
Train on 7200000 samples, validate on 1800000 samples, 1758 steps per epoch


1758it [02:05, 14.05it/s]


Epoch 1/100
137s - loss:  1.4948 - mse:  6.4381 - val_mse:  5.6393


1758it [02:02, 14.30it/s]


Epoch 2/100
135s - loss:  1.3644 - mse:  5.3214 - val_mse:  5.5551


...

Epoch 98/100
141s - loss:  1.2890 - mse:  4.6221 - val_mse:  5.5071


1758it [02:11, 13.41it/s]


Epoch 99/100
143s - loss:  1.2889 - mse:  4.6203 - val_mse:  5.5059


1758it [02:11, 13.36it/s]


Epoch 100/100
143s - loss:  1.2887 - mse:  4.6190 - val_mse:  5.4758


In [10]:
pred = model.predict(test_model_input,2048)  
sub = pd.DataFrame()  
sub["line_number"] = test_data.index  
sub["buffer_rate_prediction"] = pred/100  
sub.to_csv("cdn_qos_prediction.csv",index=False)